<a href="https://colab.research.google.com/github/ZaidAl2004/AER850_Project3/blob/main/AER850_Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Step 1: Object Masking

import cv2
import numpy as np
import matplotlib.pyplot as plt

#Load image
from google.colab import drive
drive.mount('/content/drive')

img = cv2.imread('/content/drive/MyDrive/project3data/project3data/motherboard_image.JPEG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Plotting to ensure image was called correctly
#plt.figure(figsize=(6,6))
#plt.title("Original Image")
#plt.imshow(img)
#plt.axis("off")

#Changing to gray and blurring it
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)

#Plotting grayscale image
#plt.figure(figsize=(6,6))
#plt.title("Grayscale")
#plt.imshow(gray, cmap='gray')
#plt.axis("off")


#Thresholding (Binary Mask Candidate)
_, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

#Plotting Threshold
#plt.figure(figsize=(6,6))
#plt.title("Thresholded Image")
#plt.imshow(thresh, cmap='gray')
#plt.axis("off")

#Inverting thresh
thresh = 255 - thresh

#Edge Detection (Canny)
edges = cv2.Canny(blur, 30, 80)

#Plotting Edges
#plt.figure(figsize=(6,6))
#plt.title("Canny Edges")
#plt.imshow(edges, cmap='gray')
#plt.axis("off")

#Combining Edges and Thresholding
combined = cv2.bitwise_or(thresh, edges)

#Plotting Edges
plt.figure(figsize=(6,6))
plt.title("Combined (Edges + Threshold)")
plt.imshow(combined, cmap='gray')
plt.axis("off")


#Contour Detection
#Find Contours
contours, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest = max(contours, key=cv2.contourArea)

mask=np.zeros_like(gray)
cv2.drawContours(mask, [largest], -1, 255, -1)

#Sort Contours by area
#contours = sorted(contours, key=cv2.contourArea, reverse=True)
#pcb_contour = contours[0] #Once sorted, largest contours (PCB) will be at first index

#Drawing Contours for visualization
#contour_img = img.copy()
#cv2.drawContours(contour_img, [pcb_contour], -1, (255, 0, 0), 6)

#plt.figure(figsize=(6,6))
#plt.title("PCB Contour")
#plt.imshow(contour_img)
#plt.axis("off")

#Creating Mask from Contour
#mask = np.zeros(gray.shape, dtype=np.uint8)
#cv2.drawContours(mask, [pcb_contour], -1, 255, thickness=cv2.FILLED)

#Filling in the holes
kernel = np.ones((15,15), np.uint8)
mask_closed= cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

plt.figure(figsize=(6,6))
plt.title("Final PCB Mask")
plt.imshow(mask_closed, cmap='gray')
plt.axis('off')

#Extracting the PCB mask

masked = cv2.bitwise_and(img, img, mask=mask_closed)

plt.figure(figsize=(6,6))
plt.title("Extracted PCB")
plt.imshow(masked)
plt.axis('off')



#Step 2: YOLOV11 Training

#Importing the library
#!pip install ultralytics
from ultralytics import YOLO

#Loading in the model
model = YOLO("yolo11n.pt")

#Training the model
model.train(
    data='/content/drive/MyDrive/project3data/project3data/data/data/data.yaml',
    epochs = 1,
    batch = 16,
    imgsz = 960,
    name='pcb_detector_v11'
)

#Saving the best model
model = YOLO("runs/detect/pcb_detector_v11/weights/best.pt")


#Step 3: Evaluating Model
#Testing for results
eval_results = model.predict("/content/drive/MyDrive/project3data/project3data/data/data/evaluation/")
eval_results.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/project3data/project3data/data/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic